In [1]:
import pandas as pd

%load_ext autoreload
%autoreload 2
from utils import data

In [2]:
# Allow for import of dataloader from pdf_extraction folder
import os
import sys

module_path = os.path.abspath(os.path.join("../../04_dataset_access/"))
if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import Dataloader  # type: ignore

In [3]:
if not os.path.isfile("tmp.arrow"):
    df = pd.read_csv(
        "../../24212003_requirements_for_artificial_intelligence/patched_feedbacks.csv"
    )
    df = Dataloader("sentence", tokenize=True, detect_lang=True).from_folder(
        "../../24212003_requirements_for_artificial_intelligence/attachments/", df
    )

    df["text"].str.replace("\n|\u200b", "", regex=True)

    all = df
    df = df.query("language_detected == 'en' and tokenized.str.len() > 0").copy()

    df = df.reset_index(drop=True).reset_index(names="sentence_index")
    df.reset_index(drop=True).to_feather("tmp.arrow")

else:
    df = pd.read_feather("tmp.arrow")

print(len(df))
df.head(2)

As n_jobs=-1 <= 0, enabling multiprocessing with 24 cores!
As n_jobs=-1 <= 0, enabling multiprocessing with 24 cores!
As n_jobs=-1 <= 0, enabling multiprocessing with 24 cores!
33307


,sentence_index,id,text,language,country,user_type,organization,surname,feedback,status,...,publication,publication_id,publication_status,tr_number,scope,governance_level,full_name,source,language_detected,tokenized
0,0,2665651,Equinet welcomes the opportunity to provide co...,en,BEL,ngo,Equinet,NaN,Equinet welcomes the opportunity to provide co...,PUBLISHED,...,anonymous,24212003,closed,NaN,NaN,NaN,NaN,feedback,en,"[equinet, welcome, the, opportunity, to, provi..."
1,1,2665651,Equinet further wishes to reiterate its suppor...,en,BEL,ngo,Equinet,NaN,Equinet welcomes the opportunity to provide co...,PUBLISHED,...,anonymous,24212003,closed,NaN,NaN,NaN,NaN,feedback,en,"[equinet, far, wish, to, reiterate, its, suppo..."


In [4]:
df = data.add_aspect_and_string(df)

pd.concat((df.iloc[:1], df.dropna(subset=["aspect"]).iloc[:2]))

,sentence_index,id,text,language,country,user_type,organization,surname,feedback,status,...,full_name,source,language_detected,tokenized,sentence_before,aspect,sentence_after,sentiment_string,aspect_type,aspect_subtype
0,0,2665651,Equinet welcomes the opportunity to provide co...,en,BEL,ngo,Equinet,NaN,Equinet welcomes the opportunity to provide co...,PUBLISHED,...,NaN,feedback,en,"[equinet, welcome, the, opportunity, to, provi...",,None,,NaN,None,None
33,33,2665651,Article 7 of the current Proposal defines adve...,en,BEL,ngo,Equinet,NaN,Equinet welcomes the opportunity to provide co...,PUBLISHED,...,NaN,attachment,en,"[article, 7, of, the, current, proposal, defin...",,Article 7,of the current Proposal defines adverse impac...,[CLS] Article 7 of the current Proposal defin...,article,7
76,76,2665650,AI Austria welcomes the opportunity to comment...,en,AUT,ngo,AI Austria,Gorzala,AI Austria welcomes the opportunity to comment...,PUBLISHED,...,Jeannette Gorzala,feedback,en,"[ai, austria, welcome, the, opportunity, to, c...",AI Austria welcomes the opportunity to comment...,Artificial Intelligence Act,.,[CLS]AI Austria welcomes the opportunity to co...,ai_act,


In [5]:
df = data.add_sentiment_columns(df)

/home/felix/uni/thesis/07_sentiment/sentiment/env/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Exporting the results

In [6]:
df.to_feather("../../06_explorer/explorer/data/sentiments.arrow")